In [ ]:
-- Run this script to follow along with the demo
USE [master];
GO

-- Checking to see if our database exists and if it does drop it
IF DATABASEPROPERTYEX ('ABCCompany','Version') IS NOT NULL
BEGIN
	ALTER DATABASE [ABCCompany] SET SINGLE_USER
	WITH ROLLBACK IMMEDIATE;
	DROP DATABASE [ABCCompany];
END
GO

CREATE DATABASE [ABCCompany];
GO

ALTER DATABASE [ABCCompany] SET RECOVERY SIMPLE;
GO

USE [ABCCompany];
GO

CREATE SCHEMA [Sales];
GO

CREATE TABLE [Sales].[SalesPersonLevel] (
	[Id] int identity(1,1) NOT NULL,
	[LevelName] nvarchar(500) NOT NULL,
	[CreateDate] datetime NOT NULL DEFAULT GETDATE(),
	[ModifyDate] datetime NULL
	CONSTRAINT [PK_SalesPersonLevel] PRIMARY KEY CLUSTERED ([Id]));
GO

INSERT INTO [Sales].[SalesPersonLevel] ([LevelName])
	VALUES	('President'),
			('Manager'),
			('Staff');
GO
	
CREATE TABLE [Sales].[SalesPerson] (
	[Id] int identity(1,1) NOT NULL,
	[FirstName] nvarchar(500) NOT NULL,
	[LastName] nvarchar(500) NOT NULL,
	[SalaryHr] decimal(32,2) NULL,
	[ManagerId] int NULL,
	[LevelId] int NOT NULL,
	[Email] nvarchar(500) NULL,
	[StartDate] date NOT NULL,
	[CreateDate] datetime NOT NULL DEFAULT GETDATE(),
	[ModifyDate] datetime NULL
	CONSTRAINT [PK_SalesPerson] PRIMARY KEY CLUSTERED ([Id]),
	CONSTRAINT [FK_SalesPersonLevel] FOREIGN KEY ([LevelId]) REFERENCES [Sales].[SalesPersonLevel] ([Id]),
	CONSTRAINT [FK_SalesPersonManagerId] FOREIGN KEY ([ManagerId]) REFERENCES [Sales].[SalesPerson] ([Id]));
GO

INSERT INTO [Sales].[SalesPerson] ([FirstName],[LastName],[SalaryHr],[ManagerId],[LevelId],[Email],[StartDate]) 
	VALUES	('Tom','Jones',300,1,1,'Tom.Jones@ABCCorp.com','1/5/2016'),
			('Sally','Smith',175,1,2,'Sally.Smith@ABCCorp.com','1/7/2018'),
			('Bill','House',100,2,3,'Bill.House@ABCCorp.com','1/8/2018'),
			('Karen','Knocks',100,2,3,'Karen.Knocks@ABCCorp.com','1/15/2017'),
			('Lisa','James',75,2,3,'Lisa.James@ABCCorp.com','6/1/2018'),
			('Kerrie','Friend',125,2,3,'Kerrie.Friend@ABCCorp.com','8/14/2018'),
			('Jason','Henderson',55,2,3,'Jason.Henderson@ABCCorp.com','1/14/2017'),
			('Wanda','Jones',55,2,3,'Tom.Jones@ABCCorp.com','9/1/2017'),
			('Jared','Lee',65,2,3,'Jared.Lee@ABCCorp.com','9/8/2018'),
			('Tammy','Smith',75,2,3,NULL,'2/5/2018');
GO

ALTER INDEX ALL ON [Sales].[SalesPerson] REBUILD;
GO

CREATE TABLE [Sales].[SalesTerritoryStatus] (
	[Id] int identity(1,1) NOT NULL,
	[StatusName] nvarchar(500) NOT NULL,
	[CreateDate] datetime NOT NULL DEFAULT GETDATE(),
	[ModifyDate] datetime NULL
	CONSTRAINT [PK_SalesTerritoryStatus] PRIMARY KEY CLUSTERED ([Id]));
GO

INSERT INTO [Sales].[SalesTerritoryStatus] ([StatusName])
	VALUES	('On Hold'),
			('In Progress'),
			('Closed');
GO

CREATE TABLE [Sales].[SalesTerritory] (
	[Id] int identity(1,1) NOT NULL,
	[TerritoryName] nvarchar(500) NOT NULL,
	[Group] nvarchar(500) NULL,
	[StatusId] int NOT NULL,
	[CreateDate] datetime NOT NULL DEFAULT GETDATE(),
	[ModifyDate] datetime NULL
	CONSTRAINT [PK_SalesTerritory] PRIMARY KEY CLUSTERED ([Id]),
	CONSTRAINT [FK_StatusId] FOREIGN KEY ([StatusId]) REFERENCES [Sales].[SalesTerritoryStatus] ([Id]));
GO

INSERT INTO [Sales].[SalesTerritory] ([TerritoryName],[Group],[StatusId]) 
	VALUES	('Northwest','North America',2),
			('Northeast','North America',2),
			('Southwest','North America',2),
			('Southeast','North America',1),
			('Canada','North America',3),
			('France','Europe',1),
			('Germany','Europe',2),
			('Australia','Pacific',2),
			('United Kingdom','Europe',3),
			('Spain','Europe',1);

ALTER INDEX ALL ON [Sales].[SalesTerritory] REBUILD;
GO

CREATE TABLE [Sales].[SalesOrder] (
	[Id] int identity(1,1) NOT NULL,
	[SalesPerson] int NOT NULL,
	[SalesAmount] decimal(36,2) NOT NULL,
	[SalesDate] datetime NOT NULL,
	[SalesTerritory] int NOT NULL,
	[OrderDescription] nvarchar(MAX) NULL,
	[CreateDate] datetime NOT NULL DEFAULT GETDATE(),
	[ModifyDate] datetime NULL
	CONSTRAINT [PK_SalesOrder] PRIMARY KEY CLUSTERED ([Id]),
	CONSTRAINT [FK_SalesPerson] FOREIGN KEY ([SalesPerson]) REFERENCES [Sales].[SalesPerson] ([Id]),
	CONSTRAINT [FK_SalesTerritory] FOREIGN KEY ([SalesTerritory]) REFERENCES [Sales].[SalesTerritory] ([Id]));
GO

INSERT INTO [Sales].[SalesOrder] ([SalesPerson],[SalesAmount],[SalesDate],[SalesTerritory],[OrderDescription]) 
	VALUES (1,2500,'04/05/2019',1,REPLICATE('Sales Description ',10)),
		   (2,3000,'03/02/2019',4,REPLICATE('Sales Description ',10)),
		   (3,4200,'06/02/2019',3,REPLICATE('Sales Description ',10)),
		   (4,1900,'07/01/2019',7,REPLICATE('Sales Description ',10)),
		   (7,2200,'05/15/2019',6,REPLICATE('Sales Description ',10)),
		   (9,5200,'06/03/2019',5,REPLICATE('Sales Description ',10)),
		   (10,7800,'04/13/2019',4,REPLICATE('Sales Description ',10)),
		   (3,4400,'03/23/2019',3,REPLICATE('Sales Description ',10)),
		   (5,1900,'02/15/2019',2,REPLICATE('Sales Description ',10)),
		   (5,7000,'6/09/2019',1,REPLICATE('Sales Description ',10));
GO

ALTER INDEX ALL ON [Sales].[SalesOrder] REBUILD;
GO

CREATE OR ALTER PROCEDURE [Sales].[GenerateSalesReport]
AS
SELECT SUM(so.SalesAmount) AS 'SalesAmount'
	   ,spl.LevelName AS 'Level'
	   ,CONCAT(sp.LastName,', ',sp.FirstName) AS 'FullName' 
FROM Sales.SalesPerson sp
LEFT OUTER JOIN Sales.SalesOrder so ON so.SalesPerson = sp.Id
LEFT OUTER JOIN Sales.SalesPersonLevel spl ON spl.Id = sp.LevelId
GROUP BY spl.LevelName, sp.LastName, sp.FirstName;
GO

In [ ]:
-- limiting access via procedure

-- create user account and granting access
create login ReportingSaless  with PASSWORD = 'thisisatest9002$'
create user ReportingSaless for login ReportingSaless
GRANT EXECUTE ON Sales.GenerateSalesReport  to ReportingSaless

--these two statements would give permission error because we wrapped them inside procedure and gave access to run proc only
update sales.salesorder set salesamount = NULL
select * from sales.salesorder

-- this procedure will execute fine without any errors
execute sales.GenerateSalesReport


drop login ReportingSaless
drop user ReportingSaless


In [ ]:
-- clear all the execution plans that has been in the cache so far
DBCC FREEPROCCACHE -- PLEASE DONT RUN IT IN PRODUCTION

In [ ]:
IF OBJECT_ID('Sales.Testproc','P') IS  NULL
    EXECUTE ('CREATE PROC Testproc  AS SELECT 1;')

In [ ]:
--parameters in stored procedure

--adding another column of type bit and setting its default value to 1
-- ALTER TABLE sales.salesperson add isactive bit
-- default  1 with VALUES;

--update via individual TSQL statement
-- update sales.salesperson set isactive=0 where id = 10


-- update via stored proc and reusing it 
CREATE OR ALTER PROCEDURE Sales.GenerateSalesReport
  @ManagerEmail nvarchar(500),
  @IsActive bit = 1
AS

BEGIN
	SELECT CONCAT(sp.LastName, ', ',sp.FirstName) AS 'SalesPersonName'
		   ,sp.Email AS 'SalesPersonEmail'
		   ,spl.LevelName AS 'SalesPersonLevel'
		   ,SUM(so.SalesAmount) AS 'SalesAmount'
	FROM Sales.SalesPerson sp
	LEFT OUTER JOIN Sales.SalesOrder so ON so.SalesPerson = sp.Id
	INNER JOIN Sales.SalesPerson mgr ON mgr.Id = sp.ManagerId
	INNER JOIN Sales.SalesPersonLevel spl ON sp.LevelId = spl.Id
    --dynamically setting input parameters
	WHERE mgr.Email = @ManagerEmail AND sp.IsActive = @IsActive
	GROUP BY sp.LastName, sp.FirstName, sp.Email, spl.LevelName;
END

--explicitly setting email parameter
EXECUTE Sales.GenerateSalesReport @ManagerEmail = 'Sally.Smith@ABCCorp.com';

--passing parameter values in order (in case if we have more no of parameters)
EXECUTE Sales.GenerateSalesReport 'Tom.Jones@ABCCorp.com';

-- --overriding the isActive default value
EXECUTE Sales.GenerateSalesReport @ManagerEmail = 'Sally.Smith@ABCCorp.com' ,@IsActive = 0;

In [ ]:
--parameters in stored procedure continued

--passing input parameters and returning an output value on the basis of that
CREATE or ALTER PROC Sales.ReturnSalesPersonId
    @EmployeeEmail nvarchar(500),
    @EmployeeId int OUTPUT 
AS
BEGIN
    select @EmployeeId = Id
    FROM Sales.SalesPerson
    where Email= @EmployeeEmail
END


DECLARE @EmployeeId int;

EXECUTE Sales.ReturnSalesPersonId @EmployeeEmail = 'Sally.Smith@ABCCorp.com', @EmployeeId = @EmployeeId OUTPUT

select @EmployeeId


In [ ]:
-- stored procedure are used to encapsulate the business logic so others might be aware of when using it

-- Business Rules
-- 1) create email with user first name and last name concatenated with dot in between and @ABCCompany.com at the end
-- 2) if start date is not provide, pass today date
-- 3) if level id is not provided, insert level id = 3

CREATE or ALTER PROC Sales.InsertSalesPerson
    @FirstName NVARCHAR(50),
    @LastName NVARCHAR(50),
    @SalaryHr decimal(32,2),
    @ManagerId int,
    @LevelId int = 3, --setting default value if not set any at the time of insert
    @StartDate DATE

AS
BEGIN
    SET NOCOUNT ON -- setting it to on wont return any messages

    BEGIN TRANSACTION
        DECLARE @Email NVARCHAR(50);            
        DECLARE @SalesPersonStartDate DATE;

        SELECT @Email = CONCAT(@FirstName,'.',@LastName,'@ABCCompany.com')
        SELECT @SalesPersonStartDate = ISNULL(@StartDate, GETDATE())

        INSERT into sales.SalesPerson (FirstName, LastName, SalaryHr, ManagerId,LevelId,Email,StartDate)
        VALUES (@FirstName,@LastName,@SalaryHr,@ManagerId,@LevelId,@Email,@StartDate)

    COMMIT TRANSACTION
END


EXEC Sales.InsertSalesPerson @FirstName= 'Bruce',
                             @LastName='Wayne',
                             @SalaryHr=475,
                             @ManagerId= 1,
                             @StartDate= '2019-09-12'


select * from sales.SalesPerson where Email='Bruce.Wayne@ABCCompany.com'

In [ ]:
--implementing temporary table and table variables in stored proc to hold intermediate results

-- PRE SQL 2016 Syntac to check if temp table exist
if OBJECT_ID('tempdb..#SalesOrder') IS NOT NULL
    DROP TABLE #SalesOrder
GO

-- creating a temp table and inserting values in temp table 
create table #SalesOrder (
    SalesAmount DECIMAL(36,2) , Id int
)

 INSERT INTO #SalesOrder (SalesAmount, Id)
 select SalesAmount, Id from Sales.SalesOrder --where SalesDate >= '1/1/2018' and   SalesDate < '12/31/2018'


--SQL 2016 and onwards syntax to drop table if exists
DROP TABLE IF EXISTS #SalesOrder 

In [ ]:
-- if you dont want to define the structure for temp table, it will simply inherit the structure

 DROP TABLE IF EXISTS #SalesOrderDemo2

select SalesAmount,Id into #SalesOrderDemo2 from Sales.SalesOrder --where SalesDate >= '1/1/2018' and   SalesDate <= '12/31/2018'

select * from #SalesOrderDemo2

In [ ]:
-- creating table type

CREATE TYPE SalesPersonId  AS TABLE (SalesPersonId int)
GO

CREATE or ALTER PROC Sales.SalesPersonDetail
    @SalesPersonInput SalesPersonId READONLY

AS
BEGIN
    SELECT 
        CONCAT(LastName,',',FirstName) AS FullName,
        SalaryHr as SalaryPerHour,
        StartDate as StartDate

    FROM SalesPerson sp 
   inner JOIN @SalesPersonInput spi ON sp.Id = spi.SalesPersonId
END
GO


DECLARE @SalesPersonInput SalesPersonId;

INSERT into @SalesPersonInput VALUES (1001)
INSERT into @SalesPersonInput VALUES (85)
INSERT into @SalesPersonInput VALUES (32)
INSERT into @SalesPersonInput VALUES (45)
INSERT into @SalesPersonInput VALUES (987)

--Causes SQL Server to display information about the amount of disk activity generated by Transact-SQL statements.


set STATISTICS IO ON
GO
EXEC Sales.SalesPersonDetail @SalesPersonInput
set STATISTICS IO off

In [ ]:
-- sp_executesql
-- system stored procedure that executes a string which is a TSQL statement  #dynamic SQL 
-- it also re-uses the execution plan from cache similar to standard procedure

DECLARE @Sqlcmd as NVARCHAR(1000)
DECLARE @SalesPersonEmailInput as NVARCHAR (500)

SET @SalesPersonEmailInput = 'rruce.Wayne@ABCCompany.com';

--can pass parameters in it as well
SET @SqlCmd = N'SELECT sp.LastName,
					   sp.FirstName,
					   sp.StartDate,
					   spl.LevelName
				FROM Sales.SalesPerson sp
				INNER JOIN Sales.SalesPersonLevel spl ON spl.Id = sp.LevelId
				WHERE Email = @SalesPersonEmail';

EXECUTE dbo.sp_executesql @SqlCmd, N'@SalesPersonEmail nvarchar(500)',
						  @SalesPersonEmail = @SalesPersonEmailInput;
GO

In [ ]:
-- Using exec
-- QUERY PLAN IS NOT RE-USED

DECLARE @SqlCmd AS nvarchar(1000);
DECLARE @SalesPersonEmail AS nvarchar(500);
SET @SalesPersonEmail = 'Rruce.Wayne@ABCCompany.com';

-- parameter that is passed is only being concatenated
SET @SqlCmd = N'SELECT sp.LastName,
					   sp.FirstName,
					   sp.StartDate,
					   spl.LevelName
				FROM Sales.SalesPerson sp
				INNER JOIN Sales.SalesPersonLevel spl ON spl.Id = sp.LevelId
				WHERE Email = '''+@SalesPersonEmail+'''';

EXECUTE (@SqlCmd);
GO

In [ ]:
-- query cache
SELECT cp.usecounts 'Execution Counts'
	   ,cp.size_in_bytes 'Size in Bytes'
	   ,cp.objtype 'Type'
	   ,st.text 'SQL Text'
 FROM [sys].[dm_exec_cached_plans] cp
CROSS APPLY [sys].[dm_exec_sql_text](cp.plan_handle) st
WHERE st.text like '%Sales.SalesPersonLevel%';
GO

In [ ]:
--CHECK the stats for stored procedures
-- In process executions will not show up
SELECT	
    ps.cached_time AS 'Cached',
	ps.execution_count AS 'Execution Count',
	ps.last_execution_time AS 'Last Execution Time',
	ps.last_logical_reads AS 'Last Logical Reads',
	ps.max_logical_reads AS 'Max Logical Reads',
	ps.last_logical_writes AS 'Last Logical Writes',
	(ps.last_elapsed_time / 1000) AS 'Last Elapsed Time',
	(ps.max_elapsed_time / 1000) AS 'Max Elapsed Time',
	(ps.min_elapsed_time / 1000) AS 'Min Elapsed Time'
FROM [sys].[dm_exec_procedure_stats] ps

-- filter the stored proc by name
WHERE [object_id] = object_id('Sales.SalesPersonDetail');
GO


In [ ]:
-- parameter sniffing

-- SQL server re-uses the execution plan over and over again instead of compiling it every time and ot gets compiled with the initial param values
-- later on if parameter values are changed